# Carga de Bibliotecas y definición de constantes


In [6]:
import pandas as pd
import numpy as np
import logging, sys
from logging.handlers import RotatingFileHandler
from pathlib import Path
from datetime import datetime
from pandas.api.types import is_numeric_dtype, is_bool_dtype
from pandas import CategoricalDtype

BASE_DIR = Path.cwd().resolve().parent
INTERIM = BASE_DIR / "data" / "interim"
REFERENCES = BASE_DIR / "references"
REPORTS_IV = BASE_DIR / "reports" / "iv"
LOG_DIR = INTERIM / "logs"

# Logging
LOG_FILE = LOG_DIR / "01_agrupacion_automatica.log"
fmt = "%(asctime)s | %(levelname)s | %(name)s | %(message)s"
datefmt = "%Y-%m-%d %H:%M:%S"

logger = logging.getLogger("agrupacion_auto")
logger.setLevel(logging.INFO)

file_handler = RotatingFileHandler(LOG_FILE, maxBytes=5_000_000, backupCount=5, encoding="utf-8")
file_handler.setFormatter(logging.Formatter(fmt=fmt, datefmt=datefmt))
stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setFormatter(logging.Formatter(fmt=fmt, datefmt=datefmt))

# Evitar duplicados si vuelves a ejecutar la celda
if not logger.handlers:
    logger.addHandler(file_handler)
    logger.addHandler(stream_handler)

logger.info("=== Notebook 01: Agrupación Automática (Binning / IV) ===")
logger.info(f"BASE_DIR={BASE_DIR}")
logger.info(f"INTERIM={INTERIM} | REFERENCES={REFERENCES} | REPORTS_IV={REPORTS_IV} | LOG_DIR={LOG_DIR}")


2025-11-03 19:57:49 | INFO | agrupacion_auto | === Notebook 01: Agrupación Automática (Binning / IV) ===
2025-11-03 19:57:49 | INFO | agrupacion_auto | BASE_DIR=C:\Users\PC RYU\Documents\Galileo\Maestria\Product Development\repo_proyecto_pe
2025-11-03 19:57:49 | INFO | agrupacion_auto | INTERIM=C:\Users\PC RYU\Documents\Galileo\Maestria\Product Development\repo_proyecto_pe\data\interim | REFERENCES=C:\Users\PC RYU\Documents\Galileo\Maestria\Product Development\repo_proyecto_pe\references | REPORTS_IV=C:\Users\PC RYU\Documents\Galileo\Maestria\Product Development\repo_proyecto_pe\reports\iv | LOG_DIR=C:\Users\PC RYU\Documents\Galileo\Maestria\Product Development\repo_proyecto_pe\data\interim\logs


# Funciones Auxiliares

In [25]:
def _safe_div(a, b):
    """División segura que evita división por cero."""
    return np.where(b == 0, 0, a / b)


def woe_iv_from_bins(df, target_col, bin_col):
    """
    Calcula tabla de WOE e IV para una variable ya binned.
    Usa observed=False en groupby para evitar FutureWarning.
    """
    g = df.groupby(bin_col, dropna=False, observed=False)
    agg = g[target_col].agg(['count', 'sum']).rename(columns={'sum': 'bads'})
    agg['goods'] = agg['count'] - agg['bads']

    total_bads  = agg['bads'].sum()
    total_goods = agg['goods'].sum()

    agg['dist_bad']  = _safe_div(agg['bads'].values,  total_bads if total_bads  != 0 else 1)
    agg['dist_good'] = _safe_div(agg['goods'].values, total_goods if total_goods != 0 else 1)

    ratio = _safe_div(agg['dist_bad'], agg['dist_good'])
    ratio = np.clip(ratio, 1e-10, None)  # evita log(0) / /0
    agg['woe'] = np.log(ratio)

    diff = agg['dist_bad'] - agg['dist_good']
    agg['iv_component'] = diff * agg['woe']
    iv_total = float(np.nansum(agg['iv_component'].values))

    return agg.reset_index(), iv_total


def _build_segments_from_bins(tmp, labels):
    """
    Construye catálogo de segmentos (bordes) por bin final.
    - tmp: DataFrame con columnas ['interval' (pd.Interval)]
    - labels: Serie con el bin final asignado a cada fila (bin_1, bin_2, ...)
    """
    seg_rows = []
    tmp2 = tmp.copy()
    tmp2['final_bin'] = labels

    # observed=False evita FutureWarning en futuras versiones de pandas
    for interval, grp in tmp2.groupby('interval', dropna=False, observed=False):
        if pd.isna(interval):
            continue
        left = float(interval.left) if interval.left is not None else np.nan
        right = float(interval.right) if interval.right is not None else np.nan
        final_bin = grp['final_bin'].iloc[0]
        seg_rows.append({'bin': final_bin, 'left': left, 'right': right, 'closed': 'right'})

    segs = pd.DataFrame(seg_rows).sort_values(['bin', 'left']).reset_index(drop=True)
    segs['segment'] = segs.groupby('bin', observed=False).cumcount() + 1
    return segs[['bin', 'segment', 'left', 'right', 'closed']]


def automatic_numeric_binning(s, y, max_bins=8, min_bin_pct=0.05, monotone=True):
    """
    Binning automático para variables numéricas con fusión de bins pequeños y monotonicidad.
    Devuelve: labels, tabla WOE/IV, iv_total y catálogo de segmentos (bordes) por bin.
    """
    mask_valid = s.notna()
    s_valid = s[mask_valid]
    if len(s_valid) == 0:
        labels = pd.Series(['NA'] * len(s), index=s.index)
        tbl, iv = woe_iv_from_bins(pd.DataFrame({'y': y, 'bin': labels}), 'y', 'bin')
        segs = pd.DataFrame({'bin': ['NA'], 'segment': [1], 'left': [np.nan], 'right': [np.nan], 'closed': ['right']})
        return labels, tbl, iv, segs

    # cortes por cuantiles
    q = np.linspace(0, 1, num=min(max_bins, 10) + 1)
    cuts = np.unique(np.nanpercentile(s_valid, q * 100))
    if len(cuts) <= 2:
        labels = pd.Series(['ALL'] * len(s), index=s.index)
        tbl, iv = woe_iv_from_bins(pd.DataFrame({'y': y, 'bin': labels}), 'y', 'bin')
        segs = pd.DataFrame({'bin': ['ALL'], 'segment': [1], 'left': [np.nan], 'right': [np.nan], 'closed': ['right']})
        return labels, tbl, iv, segs

    # binning inicial por intervalos
    try:
        binned = pd.cut(s, bins=np.unique(cuts), include_lowest=True, right=True)  # Interval dtype
    except Exception:
        labels = pd.Series(['ALL'] * len(s), index=s.index)
        tbl, iv = woe_iv_from_bins(pd.DataFrame({'y': y, 'bin': labels}), 'y', 'bin')
        segs = pd.DataFrame({'bin': ['ALL'], 'segment': [1], 'left': [np.nan], 'right': [np.nan], 'closed': ['right']})
        return labels, tbl, iv, segs

    # DF de trabajo con el intervalo original como string e Interval
    tmp = pd.DataFrame({'interval': binned, 'bin_str': binned.astype(str), 'y': y})

    # fusionar bins pequeños por % de cobertura
    counts = tmp['bin_str'].value_counts(normalize=True).sort_index()
    bins_order, bins_pct = counts.index.tolist(), counts.values

    merged, acc, acc_bins = [], 0.0, []
    for b, pct in zip(bins_order, bins_pct):
        acc += pct
        acc_bins.append(b)
        if acc >= min_bin_pct:
            merged.append(acc_bins)
            acc, acc_bins = 0.0, []
    if acc_bins:
        if merged:
            merged[-1].extend(acc_bins)
        else:
            merged.append(acc_bins)

    map_bin = {b: f"bin_{i+1}" for i, group in enumerate(merged) for b in group}
    labels = tmp['bin_str'].map(map_bin)

    # forzar monotonicidad por fusión de bins contiguos de menor delta
    if monotone:
        df_tmp = pd.DataFrame({'bin': labels, 'y': y})
        while True:
            tbl, _ = woe_iv_from_bins(df_tmp, 'y', 'bin')
            if len(tbl) <= 2:
                break
            bad_rate = (tbl['bads'] / tbl['count']).values
            asc, desc = np.all(np.diff(bad_rate) >= 0), np.all(np.diff(bad_rate) <= 0)
            if asc or desc:
                break
            diffs = np.abs(np.diff(bad_rate))
            j = np.argmin(diffs)
            left, right = tbl['bin'].iloc[j], tbl['bin'].iloc[j + 1]
            labels = labels.replace({right: left})
            df_tmp = pd.DataFrame({'bin': labels, 'y': y})

    # tabla WOE/IV final
    df_tmp = pd.DataFrame({'bin': labels, 'y': y})
    tbl, iv = woe_iv_from_bins(df_tmp, 'y', 'bin')

    # catálogo de segmentos por bin (pueden ser múltiples intervalos por bin tras merges)
    segs = _build_segments_from_bins(tmp, labels)

    return labels, tbl, iv, segs


def group_categorical(s, y, min_pct=0.02, top_n=None):
    """
    Agrupa categorías raras (<min_pct) en 'OTHER' o conserva solo las top_n más frecuentes.
    No requiere cambios para los warnings.
    """
    vc = s.value_counts(normalize=True, dropna=False)
    keep = set(vc[vc >= min_pct].index)
    if top_n is not None:
        keep = set(vc.sort_values(ascending=False).head(top_n).index)
    s2 = s.where(s.isin(keep), other='OTHER').fillna('OTHER')
    return s2


# Binning(agrupación por bins) de numéricas + Ranking IV

In [26]:
MIN_BIN_PCT_NUM = 0.02

iv_rows = []
num_bin_defs = {}
num_bin_segs = {}

total = len(num_cols)
logger.info(f"Iniciando binning numérico: {total} variables… (min_bin_pct={MIN_BIN_PCT_NUM:.2%})")

for i, c in enumerate(num_cols, start=1):
    try:
        labels, tbl, iv, segs = automatic_numeric_binning(
            df[c], df[TARGET], max_bins=8, min_bin_pct=MIN_BIN_PCT_NUM, monotone=True
        )
        iv_rows.append({'variable': c, 'iv': float(iv)})
        num_bin_defs[c] = tbl
        num_bin_segs[c] = segs.copy()
    except Exception as e:
        logger.warning(f"[NUM] Falló binning en {c}: {e}")

    if i % 50 == 0 or i == total:
        logger.info(f"Progreso numéricas: {i}/{total} procesadas")

iv_num_df = pd.DataFrame(iv_rows).sort_values('iv', ascending=False)
iv_num_path = REPORTS_IV / "iv_ranking_num.csv"
iv_num_df.to_csv(iv_num_path, index=False)
logger.info(f"Ranking IV (numéricas) guardado en {iv_num_path} | vars={len(iv_num_df)}")

iv_num_df.head(15)


2025-11-03 20:43:33 | INFO | agrupacion_auto | Iniciando binning numérico: 422 variables… (min_bin_pct=2.00%)
2025-11-03 20:43:33 | WARNING | agrupacion_auto | [NUM] Falló binning en 0: '0'
2025-11-03 20:43:33 | INFO | agrupacion_auto | Progreso numéricas: 50/422 procesadas
2025-11-03 20:43:34 | INFO | agrupacion_auto | Progreso numéricas: 100/422 procesadas
2025-11-03 20:43:36 | INFO | agrupacion_auto | Progreso numéricas: 150/422 procesadas
2025-11-03 20:43:38 | INFO | agrupacion_auto | Progreso numéricas: 200/422 procesadas
2025-11-03 20:43:39 | INFO | agrupacion_auto | Progreso numéricas: 250/422 procesadas
2025-11-03 20:43:41 | INFO | agrupacion_auto | Progreso numéricas: 300/422 procesadas
2025-11-03 20:43:42 | INFO | agrupacion_auto | Progreso numéricas: 350/422 procesadas
2025-11-03 20:43:44 | INFO | agrupacion_auto | Progreso numéricas: 400/422 procesadas
2025-11-03 20:43:44 | INFO | agrupacion_auto | Progreso numéricas: 422/422 procesadas
2025-11-03 20:43:44 | INFO | agrupaci

,variable,iv
26,mxdiasatramesinte,0.938415
177,rat_ven_sact,0.806807
377,saldo_vencido_12med,0.794326
405,saldo_vencido_12max,0.791479
383,rat_ven_sact_12med,0.767128
404,saldo_vencido_6max,0.764843
376,saldo_vencido_6med,0.759061
24,limmincont_e,0.751199
21,limmedcont_v,0.751199
20,limmxcont_v,0.751199


# Agrupación categóricas + ranking combinado

In [18]:
MIN_CAT_PCT = 0.02  # 2% mínimo por categoría; el resto → 'OTHER'
cat_iv_rows = []
cat_bin_defs = {}

if len(cat_cols) > 0:
    logger.info(f"Iniciando agrupación categórica: {len(cat_cols)} variables… (min_pct={MIN_CAT_PCT:.2%})")
    for c in cat_cols:
        try:
            s2 = group_categorical(df[c], df[TARGET], min_pct=MIN_CAT_PCT)
            tbl, iv = woe_iv_from_bins(pd.DataFrame({'y': df[TARGET], 'bin': s2}), 'y', 'bin')
            cat_bin_defs[c] = tbl
            cat_iv_rows.append({'variable': c, 'iv': float(iv)})
        except Exception as e:
            logger.warning(f"[CAT] Falló agrupación en {c}: {e}")
else:
    logger.info("No hay variables categóricas candidatas.")

iv_num_df = pd.read_csv(REPORTS_IV / "iv_ranking_num.csv")
iv_cat_df = pd.DataFrame(cat_iv_rows).sort_values('iv', ascending=False) if cat_iv_rows else pd.DataFrame(columns=['variable','iv'])
iv_all = pd.concat([iv_num_df, iv_cat_df], ignore_index=True).sort_values('iv', ascending=False)

iv_all_path = REPORTS_IV / "iv_ranking_all.csv"
iv_all.to_csv(iv_all_path, index=False)
logger.info(f"Ranking IV (num+cat) guardado en {iv_all_path} | vars={len(iv_all)}")

iv_all.head(20)


2025-11-03 20:22:19 | INFO | agrupacion_auto | Iniciando agrupación categórica: 2 variables… (min_pct=2.00%)
2025-11-03 20:22:19 | WARNING | agrupacion_auto | [CAT] Falló agrupación en 0: '0'
2025-11-03 20:22:19 | INFO | agrupacion_auto | Ranking IV (num+cat) guardado en C:\Users\PC RYU\Documents\Galileo\Maestria\Product Development\repo_proyecto_pe\reports\iv\iv_ranking_all.csv | vars=422


,variable,iv
0,mxdiasatramesinte,0.938415
1,rat_ven_sact,0.806807
2,saldo_vencido_12med,0.794326
3,saldo_vencido_12max,0.791479
4,rat_ven_sact_12med,0.767128
5,saldo_vencido_6max,0.764843
6,saldo_vencido_6med,0.759061
9,limmxcont_v,0.751199
14,limmincont_v,0.751199
12,salmedincicap_v,0.751199


# ExportGroup + detalle por variable

In [27]:
iv_all_path = REPORTS_IV / "iv_ranking_all.csv"
iv_all_df = pd.read_csv(iv_all_path)

rows = []
excluded_vars = []

def _rows_from_table(var, tbl, var_type, warn_small_pct=0.01):
    out = []
    total = float(tbl['count'].sum()) if 'count' in tbl else None
    if total and total > 0:
        small = tbl.loc[(tbl['count'] / total) < warn_small_pct]
        if len(small) > 0:
            logger.warning(f"[QA] {var}: {len(small)} bins con <{warn_small_pct:.0%} de cobertura.")
    for _, r in tbl.iterrows():
        out.append({
            'variable': var, 'var_type': var_type,
            'bin': r.get('bin') if 'bin' in r else r.name,
            'count': int(r['count']), 'bads': int(r['bads']), 'goods': int(r['goods']),
            'dist_bad': float(r['dist_bad']), 'dist_good': float(r['dist_good']),
            'woe': float(r['woe']), 'iv_component': float(r['iv_component'])
        })
    return out

# Construcción exportgroup (excluye variables ALL-only)
for v, tbl in num_bin_defs.items():
    if len(tbl) == 1 and str(tbl.iloc[0].get('bin','ALL')).upper() in ('ALL','BIN_1'):
        excluded_vars.append(v); continue
    rows.extend(_rows_from_table(v, tbl, 'num'))

for v, tbl in cat_bin_defs.items():
    if len(tbl) == 1 and str(tbl.iloc[0].get('bin','ALL')).upper() in ('ALL','BIN_1'):
        excluded_vars.append(v); continue
    rows.extend(_rows_from_table(v, tbl, 'cat'))

exportgroup = pd.DataFrame(rows).merge(
    iv_all_df.rename(columns={'iv':'iv_total'}), on='variable', how='left'
)

ts = datetime.now().strftime("%Y%m%d")
exportgroup_path = REFERENCES / f"exportgroup_{ts}.csv"
exportgroup.to_csv(exportgroup_path, index=False)

# Guardar detalle WOE por variable (auditoría)
saved_num = saved_cat = 0
for v, tbl in num_bin_defs.items():
    if v in excluded_vars: continue
    tbl.to_csv(REFERENCES / f"binning_num__{v}.csv", index=False); saved_num += 1
for v, tbl in cat_bin_defs.items():
    if v in excluded_vars: continue
    tbl.to_csv(REFERENCES / f"binning_cat__{v}.csv", index=False); saved_cat += 1

# Catálogo de cortes para numéricas (por segmentos)
cuts_rows = []
for v, segs in num_bin_segs.items():
    if v in excluded_vars: continue
    tmp = segs.copy()
    tmp.insert(0, 'variable', v)
    cuts_rows.append(tmp)
if cuts_rows:
    cuts_catalog = pd.concat(cuts_rows, ignore_index=True)
    cuts_catalog_path = REFERENCES / f"numeric_bins_catalog_{ts}.csv"
    cuts_catalog.to_csv(cuts_catalog_path, index=False)
    logger.info(f"Catálogo de cortes guardado: {cuts_catalog_path} | filas={len(cuts_catalog)}")
else:
    logger.info("Catálogo de cortes vacío (todas ALL-only).")

# Resumen/QC (observed=False para evitar FutureWarning)
summary = (exportgroup
           .groupby('variable', as_index=False, observed=False)
           .agg(n_bins=('bin', 'nunique'), iv_total=('iv_total', 'first')))
summary_path = REFERENCES / f"exportgroup_summary_{ts}.csv"
summary.to_csv(summary_path, index=False)

excluded_path = REFERENCES / f"exportgroup_excluidas_{ts}.csv"
pd.Series(sorted(excluded_vars), name='variable').to_csv(excluded_path, index=False)

logger.info(f"ExportGroup guardado: {exportgroup_path} | filas={len(exportgroup)} | variables={exportgroup['variable'].nunique()}")
logger.info(f"Detalle por variable guardado: num={saved_num} | cat={saved_cat}")
logger.info(f"Resumen QC: {summary_path} | Excluidas (ALL-only): {len(excluded_vars)} -> {excluded_path}")

exportgroup.head(15)


2025-11-03 20:44:14 | INFO | agrupacion_auto | Catálogo de cortes guardado: C:\Users\PC RYU\Documents\Galileo\Maestria\Product Development\repo_proyecto_pe\references\numeric_bins_catalog_20251103.csv | filas=2685
2025-11-03 20:44:14 | INFO | agrupacion_auto | ExportGroup guardado: C:\Users\PC RYU\Documents\Galileo\Maestria\Product Development\repo_proyecto_pe\references\exportgroup_20251103.csv | filas=860 | variables=376
2025-11-03 20:44:14 | INFO | agrupacion_auto | Detalle por variable guardado: num=375 | cat=1
2025-11-03 20:44:14 | INFO | agrupacion_auto | Resumen QC: C:\Users\PC RYU\Documents\Galileo\Maestria\Product Development\repo_proyecto_pe\references\exportgroup_summary_20251103.csv | Excluidas (ALL-only): 46 -> C:\Users\PC RYU\Documents\Galileo\Maestria\Product Development\repo_proyecto_pe\references\exportgroup_excluidas_20251103.csv


,variable,var_type,bin,count,bads,goods,dist_bad,dist_good,woe,iv_component,iv_total
0,num_contratos,num,bin_1,12674,639,12035,0.691558,0.725000,-0.047224,0.001579,0.005417
1,num_contratos,num,bin_2,4850,285,4565,0.308442,0.275000,0.114761,0.003838,0.005417
2,ead_cont_nov_refina,num,bin_1,1944,370,1574,0.400433,0.094819,1.440573,0.440259,0.566149
3,ead_cont_nov_refina,num,bin_5,15580,554,15026,0.599567,0.905181,-0.411927,0.125890,0.566149
4,impgaraoperaforma,num,bin_1,10562,610,9952,0.660173,0.599518,0.096376,0.005846,0.015807
5,impgaraoperaforma,num,bin_7,6962,314,6648,0.339827,0.400482,-0.164232,0.009962,0.015807
6,impgaraoperarefe,num,bin_1,10562,610,9952,0.660173,0.599518,0.096376,0.005846,0.015807
7,impgaraoperarefe,num,bin_7,6962,314,6648,0.339827,0.400482,-0.164232,0.009962,0.015807
8,impcontnovarefi,num,bin_1,869,185,684,0.200216,0.041205,1.580844,0.251373,0.575102
9,impcontnovarefi,num,bin_3,519,104,415,0.112554,0.025000,1.504558,0.131730,0.575102
